In [1]:
from Udep2Mono.binarization import BinaryDependencyTree
from Udep2Mono.polarization import PolarizationPipeline
from Udep2Mono.util import btreeToList
from copy import deepcopy

2020-12-25 01:20:13 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | gum       |
| pos       | gum       |
| lemma     | gum       |
| depparse  | gum       |
| sentiment | sstplus   |
| ner       | ontonotes |

2020-12-25 01:20:13 INFO: Use device: cpu
2020-12-25 01:20:13 INFO: Loading: tokenize
2020-12-25 01:20:13 INFO: Loading: pos
2020-12-25 01:20:14 INFO: Loading: lemma
2020-12-25 01:20:14 INFO: Loading: depparse
2020-12-25 01:20:15 INFO: Loading: sentiment
2020-12-25 01:20:17 INFO: Loading: ner
2020-12-25 01:20:17 INFO: Done loading processors!


In [28]:
from pattern.en import pluralize, singularize

# TODO: Verb Phrase Patterns
# 1.Intransitive: subject + VI
# 2.Linking: subject + VL + NP/AdjP
# 3.Transitive: subject + VT + NP
# 4.Ditransitive: subject + VD + NP(indirect) + NP(direct)
# 5.Complex Transitive: subject + VC + NP(direct) + NP/AdjP
# 6.Open clausal complement: VB + to/that VP
class PhrasalGenerator:
    def __init__(self):
        self.deptree = None
        self.annotated = None
        self.kb = {}
        self.tree_log = []
        self.sent_log = []
        self.mod_at_left = [
            "advmod", "amod", "advmod:count", "acl:relcl"]
        self.mod_at_right = []
        
        '''"advmod": self.generate_advmod,
            "advmod:count": self.generate_advmod,
            "amod": self.generate_amod,
            "appos": self.generate_inherite,
            "aux": self.generate_aux,
            "aux:pass": self.generate_aux,
            "case": self.generate_case,
            "cc": self.generate_cc,
           
            "ccomp": self.generate_ccomp,
            "compound": self.generate_inherite,
            "compound:prt": self.generate_inherite,
            "conj": self.generate_inherite,
            "cop": self.generate_inherite,
            "csubj": self.generate_nsubj,
            "csubj:pass": self.generate_nsubj,
            "dep": self.generate_dep,
            
            "discourse": self.generate_discourse,
            "expl": self.generate_expl,
            "fixed": self.generate_inherite,
            "flat": self.generate_inherite,
            "goeswith": self.generate_inherite,
            "iobj": self.generate_inherite,
            "mark": self.generate_inherite,
            "nmod": self.generate_nmod,
            "nmod:npmod": self.generate_nmod,
            "nmod:tmod": self.generate_nmod,
            "nmod:poss": self.generate_nmod_poss,
            
            "nummod": self.generate_nummod,
            "obj": self.generate_obj,
            "obl": self.generate_obj,
            "obl:npmod": self.generate_oblnpmod,
            "obl:tmod": self.generate_inherite,
            "parataxis": self.generate_inherite,
            "xcomp": self.generate_obj,'''

    def deptree_generate(self, tree, annotated):
        self.tree_log = []
        self.sent_log = []
        self.deptree = tree
        self.annotated = annotated
        self.generate(self.deptree)

    def generate(self, tree):
        if tree.val in self.mod_at_left:
            self.left_modifier_generate(tree)
        elif tree.left != "N":
            self.generate_default(tree)

    def delete_left_modifier(self, tree):
        tree.val = tree.right.val
        tree.mark = tree.right.mark
        tree.npos = tree.right.npos
        tree.id = tree.right.id
        tree.left = tree.right.left
        tree.right = tree.right.right

    def rollback(self, tree, backup):
        tree.val = backup.val
        tree.left = deepcopy(backup.left)
        tree.right = deepcopy(backup.right)
        tree.mark = backup.mark
        tree.npos = backup.npos
        tree.id = backup.id

    def left_modifier_generate(self, tree):
        # adv + VB | VB + adv => VB
        # amod + Noun => Noun
        # Noun + relcl => Noun
        left = tree.left
        right = tree.right
        backup = deepcopy(tree)

        if right.mark == "+":
            self.delete_left_modifier(tree)
            self.save_tree(isTree=True)
            self.rollback(tree, backup)    

    def rollback_annotation(self, generated, original):
        word_id = self.annotated[generated]
        del self.annotated[generated]
        self.annotated[original] = word_id

    def generate_default(self, tree):
        left = tree.left
        right = tree.right

        if right.npos is not None:
            if "NN" in right.npos and right.mark == "-":
                for adj in kb["ADJ"]:
                    generated = ' '. join([adj, right.val])
                    word_id = self.annotated[right.val]
                    del self.annotated[right.val]
                    self.annotated[generated] = word_id
                    self.save_tree(isTree=False)
                    self.rollback_annotation(generated, right.val)
                for rel in kb["RCL"]:
                    generated = ' '. join([right.val, rel])
                    word_id = self.annotated[right.val]
                    del self.annotated[right.val]
                    self.annotated[generated] = word_id
                    self.save_tree(isTree=False)
                    self.rollback_annotation(generated, right.val)
            elif "VB" in right.npos and right.mark == "-":
                for adj in kb["ADV"]:
                    generated = ' '. join([adv, right.val])
                    word_id = self.annotated[right.val]
                    del self.annotated[right.val]
                    self.annotated[generated] = word_id
                    self.save_tree(isTree=False)
                    self.rollback_annotation(generated, right.val)

        self.generate(left)
        self.generate(right)  

    def save_tree(self, isTree):
        if isTree:
            self.tree_log.append(deepcopy(self.deptree))
        else:
            self.sent_log.append(deepcopy(self.annotated))
        
    def buildTree(self, config):
        left = BinaryDependencyTree(
            config['mod'], "N", "N", 1024, 
            wid=config['lid'], npos="JJ")
        right = BinaryDependencyTree(
            config['head'], "N", "N", 1024,
            wid=config['rid'], npos="NN")
        tree = BinaryDependencyTree(config['rel'], left, right, 1025)
        left.mark = config['mark']
        right.mark = config['mark']
        tree.mark = config['mark']
        return tree

In [31]:
sentences = ["Some red flowers need light", "All flowers need light"]
kb = {"ADJ": ["beautiful", "red", "fragret"], 
      "ADV": ["ergently", "clearly", "neccesaraly"],
      "RCL": ["which is beautiful", "which opens at night"]}

pipeline = PolarizationPipeline(sentences, verbose=2, parser="stanza")
pipeline.run_polarize_pipeline()
print("\nPolarization Complete")

phrasalGenerator = PhrasalGenerator()
for annotation in pipeline.annotations:
    print("================")
    print("Init Premise: ", annotation['annotated'])
    phrasalGenerator.kb = kb
    phrasalGenerator.deptree_generate(
        annotation['polarized_tree'], annotation['word_dict'])
    for gen_tree in phrasalGenerator.tree_log:
        generated, queue, _, _ = btreeToList(gen_tree, len(annotation['original']), {}, 0)
        annotated = list(queue.popkeys())
        print("Next Premise: ", ' '.join(annotated))
    for gen_sent in phrasalGenerator.sent_log:
        print("Next Premise: ", ' '.join(gen_sent.popkeys()))
    

100%|██████████| 2/2 [00:00<00:00,  6.89it/s]
Polarization Complete
Start Premise:  some↑ red↑ flowers↑ need↑ light↑
Next Premise:  some↑ flowers↑ need↑ light↑
Start Premise:  all↑ flowers↓ need↑ light↑
Next Premise:  all beautiful flowers need light
Next Premise:  all red flowers need light
Next Premise:  all fragret flowers need light
Next Premise:  all flowers which is beautiful need light
Next Premise:  all flowers which opens at night need light

